<a href="https://colab.research.google.com/github/DkMaria/Fake-News-Detection/blob/main/transe_no2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

KGE1401 with no the information of label

In [ ]:
pip uninstall tensorflow

In [ ]:
pip install "tensorflow>=1.15.2,<2.0"

In [3]:
# install AmpliGraph and other dependencies

%%capture 
# Install AmpliGraph library
! pip install ampligraph

# Required to visualize embeddings with tensorboard projector, comment out if not required!
! pip install --user tensorboard

# Required to plot text on embedding clusters, comment out if not required!
! pip install --user git+https://github.com/Phlya/adjustText

In [ ]:
#check if tensorflow is correctly installed 
import tensorflow as tf 
print(tf.version.VERSION)

In [ ]:
# All imports used in this notebook
%tensorflow_version 1.x
import ampligraph
import numpy as np
import pandas as pd
import pandas as pd
from ampligraph.datasets import load_from_csv 
from ampligraph.evaluation import train_test_split_no_unseen, evaluate_performance, mr_score, mrr_score, hits_at_n_score
from ampligraph.latent_features import TransE
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
from ampligraph.utils import create_tensorboard_visualizations 

In [7]:
# load data
true = load_from_csv('.', '/content/drive/MyDrive/results15.csv', sep=',') 
false = load_from_csv('.', '/content/drive/MyDrive/resultsf15.csv', sep = ',')

df_true = pd.DataFrame(true, columns = ['id', 'text', 'ratingName', 'author',  'headline', 'named_entities_claim', 'named_entities_article', 'keywords', 'source', 'sourceURL', 'link', 'language'])
df_false = pd.DataFrame(false, columns = ['id', 'text', 'ratingName', 'author', 'headline', 'named_entities_claim', 'named_entities_article', 'keywords', 'source', 'sourceURL', 'link', 'language'])

df_true['label'] = 0
df_false['label'] = 1

data = pd.concat([df_false, df_true])
# shuffle the dataset
data = data.sample(frac=1)

# drop data with no additional information
data = data.drop(columns= 'ratingName')
data = data.drop(columns= 'sourceURL')
data = data.drop(columns = 'link')
data.head()

,id,text,author,headline,named_entities_claim,named_entities_article,keywords,source,language,label
5752,http://data.gesis.org/claimskg/claim_review/7b...,Former NFL quarterback Tim Tebow knelt on the ...,Unknown,Did Tim Tebow Kneel During the National Anthem...,"2009 BCS Championship Game,American national a...","Tim Tebow,quarterback",NaN,snopes,English,1
1971,http://data.gesis.org/claimskg/claim_review/0e...,'Trump Management was charged with discriminat...,Hillary Clinton,Clinton ad: Trump Management was charged with ...,"Art of the Deal,Civil Rights Act of 1968,Donal...",NaN,"Candidate Biography,Civil Rights,Housing,Legal...",politifact,English,0
5597,http://data.gesis.org/claimskg/claim_review/90...,'...the earth moves closer to the sun every ye...,Scott Wagner,PA GOPer’s climate change theory debunked: Nop...,"Drexel University,NPR,Pennsylvania Democratic ...",NaN,Climate Change,politifact,English,1
2356,http://data.gesis.org/claimskg/claim_review/b0...,"Says 'in this next biennium, the cost of prima...",John Kitzhaber,Are increased PERS costs responsible for a $50...,"2007 stock market crash,John Kitzhaber,Oregon ...",NaN,Education,politifact,English,0
7790,http://data.gesis.org/claimskg/claim_review/a4...,'They didn’t even put up a fight against SALT',Donald Trump,Trump says NY didn't fight SALT cap,"Adirondack,Albany,Andrew Cuomo,Catskills,Charl...",NaN,Taxes,politifact,English,1


In [8]:
#knowledge graph creation
# naming entities
data['id'] =  data.id 
data['text'] = data.text 
data['author'] = data.author 
data['headline'] = data.headline 
data['named_entities_claim'] = data.named_entities_claim  
data['named_entities_article'] =  data.named_entities_article 
data['keywords'] =  data.keywords 
data['source'] = data.source 
data['language'] = data.language 
#data['label'] = data.label

In [9]:
triples = []
for _, row in data.iterrows():
    text = (row['id'], "hasText", row['text'])
    author = (row['id'], "isWrittenBy", row['author'])
    headline = (row['id'], "hasHeadline", row['headline'])
    named_entities_claim = (row['id'], "referredEntities", row["named_entities_claim"])
    keywords = (row['id'], "containKeywords", row['keywords'])
    language = (row['id'], "isWrittenIn", row['language'])
    source = (row['id'], "isLocated", row['source'])
    #label = (row['id'], "isLabelled", row['label'])

    triples.extend((text, author, headline, named_entities_claim, keywords, language, source))

In [10]:
# split triples to train-test set

X_train, X_test = train_test_split_no_unseen(np.array(triples), test_size=10000)
print('Train set size: ', X_train.shape)
print('Test set size: ', X_test.shape)

Train set size:  (91178, 3)
Test set size:  (10000, 3)


In [11]:
model = TransE(k=150,                                                             # embedding size
               epochs=100,                                                        # Num of epochs
               batches_count= 10,                                                 # Number of batches 
               eta=1,                                                             # number of corruptions to generate during training
               loss='pairwise', loss_params={'margin': 1},                        # loss type and it's hyperparameters         
               initializer='xavier', initializer_params={'uniform': False},       # initializer type and it's hyperparameters
               regularizer='LP', regularizer_params= {'lambda': 0.001, 'p': 3},   # regularizer along with its hyperparameters
               optimizer= 'adam', optimizer_params= {'lr': 0.001},                # optimizer to use along with its hyperparameters
               seed= 0, verbose=True)
tf.logging.set_verbosity(tf.logging.ERROR)
model.fit(X_train)

Average TransE Loss:   0.024256: 100%|██████████| 100/100 [03:18<00:00,  1.99s/epoch]


In [12]:
# Evaluating knowledge embeddings
print('Size of X_test:', X_test.shape)

# Append the Train, Valid and Test set to the X_filter.
''' The model has not "observed" them during training. We do so because, we would like
   to evaluate a test triple against it's corruptions and not against known facts. 
   If we know that the Validation triples and Test triples are facts (and not queries),
   we need to filter these triples out of the generated corruptions. This is the standard 
   procedure that is used to compute the metrics to compete on the leadership board.'''

X_filter = np.concatenate([X_train, X_test], 0)

ranks = evaluate_performance(X_test, 
                             model=model,
                             filter_triples=X_filter)

mr = mr_score(ranks)
mrr = mrr_score(ranks)

print("MRR: %.2f" % (mrr))
print("MR: %.2f" % (mr))

hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))

Size of X_test: (10000, 3)
WARNING - You are attempting to use 66055 distinct entities to generate synthetic negatives in the evaluation
    protocol. This may be unnecessary and will lead to a 'harder' task. Besides, it will lead to a much slower
    evaluation procedure. We recommended to set the 'corruption_entities' argument to a reasonably sized set
    of entities. The size of corruption_entities depends on your domain-specific task.


/usr/local/lib/python3.7/dist-packages/ampligraph/evaluation/protocol.py:952: UserWarning: You are attempting to use 66055 distinct entities to generate synthetic negatives in the evaluation
    protocol. This may be unnecessary and will lead to a 'harder' task. Besides, it will lead to a much slower
    evaluation procedure. We recommended to set the 'corruption_entities' argument to a reasonably sized set
    of entities. The size of corruption_entities depends on your domain-specific task.
  warnings.warn(warn_msg % ent_for_corruption_size)
100%|██████████| 10000/10000 [19:52<00:00,  8.38it/s]


MRR: 0.35
MR: 2330.76
Hits@10: 0.43
Hits@3: 0.38
Hits@1: 0.31


In [13]:
from sklearn.model_selection import train_test_split

y = data['label']
X1_train, X1_test, y1_train, y1_test = train_test_split(data['id'], y, test_size=0.20, random_state=42)

print('X1_train:', len(X1_train))
print('y1_train:', len(y1_train))
print('****')
print('X1_test:', len(X1_test))
print('y1_train:', len(y1_test))

X1_train: 11563
y1_train: 11563
****
X1_test: 2891
y1_train: 2891


In [14]:
# get the embeddings for X1_train
X_train_clf = model.get_embeddings(X1_train)
X_test_clf = model.get_embeddings(X1_test)

In [15]:
# XGB classifier
clf_model = XGBClassifier(n_estimators=500, max_depth=5, binary="logistic")

clf_model.fit(X_train_clf,y1_train)

print('Classification report for TransE - XGBclassifier without label information')
print(classification_report(y1_test,  clf_model.predict(X_test_clf)))

Classification report for TransE - XGBclassifier without label information
              precision    recall  f1-score   support

           0       0.46      0.14      0.21       904
           1       0.70      0.93      0.80      1987

    accuracy                           0.68      2891
   macro avg       0.58      0.53      0.51      2891
weighted avg       0.63      0.68      0.62      2891



In [16]:
create_tensorboard_visualizations(model, 'fn_embeddings_transeNo')